In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pickle
import json
import copy
import random
from pathlib import Path
sys.path.append('../lared')
from data_loading.utils import get_video_caps, AudioFiles
from dataset.example import VideoExample, AudioExample

In [3]:
from tqdm.notebook import tqdm

In [4]:
from covfee.orm import Project

In [5]:
outpath = '/home/jose/furnace/covfee-annotation'

In [6]:
cloud_data_path = '/home/jose/drive/data/lared_laughter'
VideoExample.init_caps(get_video_caps('/mnt/hdd/data/lared/concat'))
audio_ds = AudioFiles('/mnt/hdd/data/lared/audio/trimmed')
AudioExample.init_dataset(audio_ds)
random.seed('25')

In [7]:
examples = pickle.load(open(cloud_data_path + '/examples/examples.pkl', 'rb'))

In [8]:
examples[0].__dict__

{'id': 0,
 'pid': 1,
 'cam': 2,
 'valid': None,
 'ini_time': 1023.28,
 'fin_time': 1033.08,
 'labels': {'vad': None, 'label': None, 'vad_seg': None},
 'rect': [273, 216, 170, 205]}

In [9]:
examples[0].get_file_id()

'1_0_cam_2'

In [10]:
task = {
    "type": "QuestionnaireTask",
    "form": {
        "fields": [
            {
                "prompt": "Level of enjoyment: ",
                "input": {
                  "type": "Slider",
                  "defaultValue": 50
                }
            }
        ]
    },
    "media": {
        "type": "video",
        "autoplay": False,
        "controls": True
    }
}

In [11]:
examples = random.sample(examples, 20)

In [12]:
tasks = []
output_path = os.path.join(cloud_data_path, 'pilot')
audio_path = os.path.join(output_path, 'audio')
videos_path = os.path.join(output_path, 'video')
av_path = os.path.join(output_path, 'av')

for example in tqdm(examples):
    t = copy.deepcopy(task)
    t['media']['url'] = 'av/' + example.get_file_id() + '.mp4'
    tasks.append(t)
    # write output files
    example.write_video(videos_path)
    example.write_audio(audio_path)
    example.write_audiovisual(video_path=videos_path, audio_path=audio_path, fout_path=av_path)

/mnt/hdd/data/lared/audio/trimmed/chan_01_2.wav
/mnt/hdd/data/lared/audio/trimmed/chan_02_2.wav
/mnt/hdd/data/lared/audio/trimmed/chan_01_2.wav
/mnt/hdd/data/lared/audio/trimmed/chan_02_2.wav



In [13]:
project = {
    "name": "Pilot",
    "email": "example@example.com",
    "hits": [
        {
            "type": "timeline",
            "tasks": [
                {
                    "type": "VideoInstructionsTask"
                },
                *tasks
            ]
                
        }
    ]
}

In [14]:
json.dump(project, open(os.path.join(outpath, 'annot_visual_enjoyment.covfee.json'), 'w'), indent=2)